In [71]:
%run data_processor.py

clean the text: 3108it [00:00, 30984.29it/s]

mr


clean the text: 10662it [00:00, 29505.14it/s]
10662it [00:00, 26543.20it/s]
clean the text: 384it [00:00, 3838.34it/s]

Average length: 20.38585631213656
doc count: 10662
Total number of words: 18764
ohsumed


clean the text: 7400it [00:01, 4135.52it/s]
7400it [00:02, 3458.18it/s]
clean the text: 1384it [00:00, 6566.94it/s]

Average length: 135.8177027027027
doc count: 7400
Total number of words: 14157
R52


clean the text: 9100it [00:01, 8139.41it/s]
9100it [00:01, 6400.41it/s]
clean the text: 968it [00:00, 9676.85it/s]

Average length: 69.81989010989011
doc count: 9100
Total number of words: 8892
R8


clean the text: 7674it [00:00, 9345.78it/s]
7674it [00:01, 6018.99it/s]

Average length: 65.72126661454261
doc count: 7674
Total number of words: 7688


In [72]:
%run build_graph.py


==> Dataset:mr<==


generate tfidf edge: 1993it [00:00, 10089.06it/s]

tfidf time: 0.362199068069458
tfidf_vec shape: (10662, 18764)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 18764


generate tfidf edge: 10662it [00:01, 9648.84it/s]
Split by window:   6%|███▍                                                       | 628/10662 [00:00<00:03, 3241.27it/s]

+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  29426  |  196826  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+
pmi read file len: 10662


...

tfidf time: 1.0448355674743652
tfidf_vec shape: (7674, 7688)
tfidf_vec type: <class 'scipy.sparse.csr.csr_matrix'>
vocab_lst len: 7688


generate tfidf edge: 7674it [00:02, 3349.63it/s]
Split by window:   0%|                                                                        | 0/7674 [00:00<?, ?it/s]

+---------+----------+-------------+------------+----------+
|  Nodes  |  Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+----------+-------------+------------+----------+
|  15362  |  323670  |      0      |     0      |  1.0000  |
+---------+----------+-------------+------------+----------+
pmi read file len: 7674


Calculate pmi between words: 100%|████████████████████████████████████████| 2329566/2329566 [00:29<00:00, 79869.64it/s]


Total number of edges between word: 1578214
pmi time: 76.35704135894775
+---------+-----------+-------------+------------+----------+
|  Nodes  |   Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+-----------+-------------+------------+----------+
|  15362  |  1673322  |      0      |     0      |  1.0000  |
+---------+-----------+-------------+------------+----------+
total time: 77.40187692642212




In [8]:
%%time
# Vanilla
%run trainer.py


 mr
Namespace(data_path='./data', dataset='mr', device=device(type='cpu'), dropout=0.2, early_stopping=10, graph_path='../tmp/TCGCN/graph', lr=0.02, max_epoch=50, nhid=200, tmp_path='../tmp/TCGCN', use_gf=False, val_ratio=0.1)
prepare data
Macro F1: [0.7647962399131566, 0.7676407489025916]
Namespace(data_path='./data', dataset='mr', device=device(type='cpu'), dropout=0.2, early_stopping=10, graph_path='../tmp/TCGCN/graph', lr=0.02, max_epoch=50, nhid=200, tmp_path='../tmp/TCGCN', use_gf=True, val_ratio=0.1)
prepare data
EC:  [1.0000183805081024, 1.0001191303868895, 1.0000481796759189, 1.001212702389186, 1.0000437277443386, 1.0000041684843928, 1.0000067119030618, 1.000068711082344, 1.000125334986635, 1.0000245219744561]


RuntimeError: indices and values must have same nnz, but got nnz from indices: 29426, nnz from values: 29424

Wall time: 1min 19s


In [68]:
%%time
# EC
%run trainer.py

Namespace(data_path='./data', dataset='R8', device=device(type='cpu'), dropout=0.2, early_stopping=10, graph_path='../tmp/TCGCN/graph', lr=0.02, max_epoch=50, nhid=200, tmp_path='../tmp/TCGCN', use_gf=True, val_ratio=0.1)
prepare data
+---------+-----------+-------------+------------+----------+
|  Nodes  |   Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+-----------+-------------+------------+----------+
|  15362  |  1673322  |      0      |     0      |  1.0000  |
+---------+-----------+-------------+------------+----------+
Is subgraph connected:  True
+---------+-----------+-------------+------------+----------+
|  Nodes  |   Edges   |  Selfloops  |  Isolates  |  覆盖度  |
+---------+-----------+-------------+------------+----------+
|   7688  |  1349652  |      0      |     0      |  1.0000  |
+---------+-----------+-------------+------------+----------+
[0.20979919205544756, 0.26159571491160943, 0.26363128309917433, 0.4427468067106428, 0.30478138343953814, 0.425872313040481

In [9]:
g_info = pickle.load(open("../tmp/TCGCN/graph/mr_g_info.pkl", 'rb'))
bc = g_info["bc_subg"]
dict1 = OrderedDict(sorted(bc.items()))
len(dict1)

18762

In [6]:
g_info

{'num_docs': 10662,
 'num_words': 18764,
 'bc_subg': {10662: 0.0012641204833020467,
  10663: 0.006317058187812721,
  10664: 1.728773363806138e-06,
  10665: 6.323401007701976e-06,
  10666: 1.0525416755355659e-05,
  10667: 9.923173262990699e-06,
  10668: 4.416455678636162e-06,
  10669: 5.3189754023177726e-05,
  10670: 0.0,
  10671: 3.053589168517849e-05,
  10672: 2.004721341407781e-06,
  10673: 0.00015429170857282843,
  10674: 1.858651822613829e-05,
  10675: 6.871367861845565e-06,
  10676: 3.295451631893795e-06,
  10677: 3.100149610094611e-05,
  10678: 1.17080260555411e-05,
  10679: 6.182253474567951e-06,
  10680: 4.7794878089877585e-06,
  10681: 0.0,
  10682: 1.5617112545298733e-06,
  10683: 1.1134430775930053e-06,
  10684: 7.573102172300807e-07,
  10685: 1.873354928574709e-06,
  10686: 1.6265709663211961e-06,
  10687: 1.731233356639725e-07,
  10688: 2.5852142118630756e-06,
  10689: 6.772654357346578e-06,
  10690: 6.211311582476831e-05,
  10691: 6.415703350199226e-07,
  10692: 2.3594268

In [ ]:
G.node_num
# subgraph = G.g.subgraph(np.arange(5))
# subgraph.nodes()


In [ ]:
%run data_processor.py

In [ ]:
%run build_graph.py

In [ ]:
def b_c(G):
    vertices = G.nodes()
    new_bc = {}
    paths = defaultdict(dict)

    # Get shortest paths between all pairs of vertices
    for i, vertex in enumerate(vertices[:-1]):
        for o_vertex in vertices[i+1:]:
            paths[vertex][o_vertex] = [path for path in
                                       nx.all_shortest_paths(G, vertex, o_vertex)]

    for vertex in vertices:
        counter = 0
        for i, vertex1 in enumerate(vertices[:-1]):
            for vertex2 in vertices[i+1:]:
                for path in paths[vertex1][vertex2]:
                    if vertex in path[1:-1]:
                        counter += 1
        new_bc[vertex] = counter

    return new_bc

In [ ]:
vertices = G.g.nodes()
len(vertices), vertices
# for i, vertex in enumerate(vertices[:-1]):
#     print(i)

In [ ]:
import torch as th
import numpy as np
value = [10,20,30]#[1.] * 3
shape = (3, 3)
indices = th.from_numpy(
        np.vstack((list(range(3)), list(range(3)))).astype(np.int64))
values = th.FloatTensor(value)
shape = th.Size(shape)
features = th.sparse.FloatTensor(indices, values, shape)
features

In [ ]:
from networkx.algorithms.centrality import eigenvector_centrality
ec = eigenvector_centrality(G.g)

# موارد زیر جواب ندادن
# closeness_centrality
#betweenness_centrality
# bc = betweenness_centrality(G.g, k =1000)

# bc = b_c(G.g)

In [ ]:
# len(ec), ec[1]
# type(ec)
# ec.value
# Creates a sorted dictionary (sorted by key)
from collections import OrderedDict

# dict = {'ravi':'10','rajnish':'9','sanjeev':'15','yash':'2','suraj':'32'}
dict1 = OrderedDict(sorted(ec.items()))
# print(dict1)
# ec_list = dict1.values()
ec_list = [x for x in dict1.values()]
# list(ec_list)
np.min(ec_list), np.max(ec_list), np.mean(ec_list)

In [ ]:
ec_list[:4]

In [ ]:
# len(G.g.nodes())
G.g[0]

In [ ]:
pos = nx.nx_pydot.pydot_layout(G.g)
plt.figure()    
nx.draw(G.g,pos,edge_color='black',width=1,linewidths=1, node_size=10,node_color='blue',alpha=0.9)
plt.axis('on')
plt.show()

In [ ]:
import networkx as nx 
import matplotlib.pyplot as plt 

fig = plt.figure(figsize=(40, 40)) 
# G = nx.fast_gnp_random_graph(300, 0.02, seed=1337) 
nx.draw(G.g, node_size=30) 
plt.axis('equal') 
plt.show() 
fig.savefig('waka.svg')